# Create Docker Container and Deploy dacpac Test

We need to create a repeatable test scenario to test the deployment of our database project.

This involves setting up a docker container that we can have as the target.

Steps:

1. Confirm if the docker container exists
2. If not, create 
3. Start the docker container
4. Use dbatools to create and deploy the dacpac to the container

Set the parameters for the container:

In [ ]:
$containername = 'PracticalStarterGuide'
$port = "2022:1433" # the first value is the port we connect to. 1433 is the port internally to docker
$password = 'Ch@ngeM3S00n!'
$dacpaclocation = 'C:\GitRepo\SQL_Presentations\A Practical Starter Guide to SQL Notebooks\SuperheroesDacPac\Superheroes.dacpac'


Create the secure credential to be used below:

Borrowing heavily from [https://adamtheautomator.com/powershell-get-credential/](https://adamtheautomator.com/powershell-get-credential/)

In [ ]:
$newpassword = ConvertTo-SecureString $password -AsPlainText -Force
$credential = New-Object System.Management.Automation.PSCredential ('sa', $newpassword)

Start by checking to see if the container exists

In [ ]:
[System.Collections.ArrayList]$allcontainers = @()

# for all the containers....
docker container ls --format '{{.Names}}' -a | ForEach-Object {
    $cname = $_

    # get the status
    $cstatus = docker ps -f "name=$cname" --format '{{.Names}}'
    if ($cstatus) {$cstatus = $true} else {$cstatus = $false}

    # get the port
    $cport = docker container inspect "$cname" --format '{{.HostConfig.PortBindings}}'
    $cport = $cport.substring($cport.length - 7, 4)

    # add them to the array
    $allcontainersval = [PSCustomObject]@{'ContainerName'=$cname;'Port'=$cport;'ContainerRunning'=$cstatus}
    $allcontainers.add($allcontainersval) | Out-Null
    
}

$allcontainers

Check to see if we need to create the specific container

In [ ]:
if ($allcontainers | where {$_.ContainerName -eq $ContainerName}) {
    $CreateContainer = $false

    #see if it's running
    $isrunning = $allcontainers | Where {$_.ContainerName -eq $ContainerName} | SELECT ContainerRunning

    #if the container is started, set the flag to not start it and end other work
    if ($isrunning -eq $true) {
        $stopProcess = $true
        Write-Host "$ContainerName container exists and is already running."
    }
    else {
        $StartContainer = $true
        $stopProcess = $false
        Write-Host "$ContainerName container exists but is not running"
    }
} else {
    $CreateContainer = $true
    $StartContainer = $true
    $stopProcess = $false
    Write-Host "$ContainerName container does not exist and needs to be created."
}

Create the container if needed

In [ ]:
if ($CreateContainer -eq $true) {
    docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=$password" `
        -p $port --name $containername -h $containername `
        -d mcr.microsoft.com/mssql/server:2022-latest
    
    Write-Host "Container $ContainerName created and started"
    
}

Start the container if needed

In [ ]:
if ($StartContainer -eq $true) {
    docker container start $containername

    Write-Host "Starting container $ContainerName"
}

Make sure the dacpac is there by creating it from the database

In [ ]:
Export-DbaDacPackage -SqlInstance debthedba\sql2019 -Database Superheroes -FilePath $dacpaclocation

Deploy dacpac to the docker container using dbatools:

Taking this from the first example from the dbatools [Publish-DbaDacPackage](https://docs.dbatools.io/Publish-DbaDacPackage) information page directly.

In [ ]:
Start-Sleep -Seconds 10

$options = New-DbaDacOption -Type Dacpac -Action Publish
$options.DeployOptions.DropObjectsNotInSource = $true
Publish-DbaDacPackage -SqlInstance 'host.docker.internal,2022' -Database Superheroes -DacOption $options -Path $dacpaclocation -SqlCredential $credential